#  Преобразование текстов в последовательность индексов токенов. 🤗 Tokenizers

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://huggingface.co/docs/tokenizers/index
* https://huggingface.co/docs/tokenizers/pipeline
* https://huggingface.co/docs/tokenizers/api/trainers#tokenizers.trainers.WordLevelTrainer
* Хороший минималистичный пакет с набором готовых преобразований, но больше не развивается:
    * https://pytorch.org/text/stable/
    * https://pytorch.org/text/stable/vocab.html
    * https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [ ]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt_tab')
word_tokenize("")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[]

In [ ]:
corpus_tokens = [
    word_tokenize(doc.lower())
    for doc in corpus
]
corpus_tokens

[['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 ['студенты',
  'активно',
  'участвуют',
  'в',
  'общественной',
  'жизни',
  'университета']]

In [ ]:
words = set()
words.update(corpus_tokens[0])
words.update(corpus_tokens[1])
words

{'активно',
 'в',
 'достичь',
 'жизни',
 'занимаются',
 'знания',
 'и',
 'общественной',
 'получить',
 'стремясь',
 'студенты',
 'университета',
 'усердно',
 'успеха',
 'участвуют'}

In [ ]:
stoi = {w: idx for idx, w in enumerate(words)}
stoi["<UNK>"] = len(stoi)
stoi["<PAD>"] = len(stoi)
stoi
# itos

{'занимаются': 0,
 'получить': 1,
 'в': 2,
 'университета': 3,
 'и': 4,
 'достичь': 5,
 'студенты': 6,
 'усердно': 7,
 'успеха': 8,
 'жизни': 9,
 'активно': 10,
 'стремясь': 11,
 'знания': 12,
 'общественной': 13,
 'участвуют': 14,
 '<UNK>': 15,
 '<PAD>': 16}

In [ ]:
stoi["усердно"]

7

In [ ]:
[stoi[w] for w in corpus_tokens[0]]

[6, 7, 0, 11, 1, 12, 4, 5, 8]

In [ ]:
corpus_i = [
    [stoi[w] for w in doc]
    for doc in corpus_tokens
]
corpus_i

[[6, 7, 0, 11, 1, 12, 4, 5, 8], [6, 10, 14, 2, 13, 9, 3]]

In [ ]:
# stoi["милана"]
stoi.get("милана", stoi["<UNK>"])

15

In [ ]:
import torch as th

In [ ]:
# padding
corpus_i[1].extend((stoi["<PAD>"], stoi["<PAD>"]))

In [ ]:
th.tensor(corpus_i)

tensor([[ 6,  7,  0, 11,  1, 12,  4,  5,  8],
        [ 6, 10, 14,  2, 13,  9,  3, 16, 16]])

2\. Обсудите основные требования к инструменту для построения набора индексов

3\. Рассмотрите пример работы с пакетом `tokenizers` для построения набора индексов токенов.

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordLevelTrainer

In [ ]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [ ]:
model = WordLevel()
tokenizer = Tokenizer(model)
tokenizer.normalizer = Lowercase()
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=["<PAD>", "<UNK>"])
tokenizer.train_from_iterator(corpus, trainer=trainer)

In [ ]:
tokenizer.get_vocab()

{'общественной': 10,
 'получить': 11,
 'достичь': 5,
 'студенты': 2,
 'жизни': 6,
 'активно': 3,
 'занимаются': 7,
 '<UNK>': 1,
 'успеха': 15,
 'в': 4,
 'университета': 13,
 'знания': 8,
 'участвуют': 16,
 'усердно': 14,
 '<PAD>': 0,
 'и': 9,
 'стремясь': 12}

In [ ]:
encoding = tokenizer.encode(corpus[0])

encoding.tokens, encoding.ids

(['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 [2, 14, 7, 12, 11, 8, 9, 5, 15])

## Задачи для самостоятельного решения

### 1.

<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть <UNK>)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран.

- [ ] Проверено на семинаре

In [ ]:
class Vocab:
    def __init__(self, data):
      self.spec_tokens = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']
      self.words = set()
      self.words.update(self.spec_tokens)

      for doc in data:
        self.words.update(doc)

      self.stoi_ = {w: idx for idx, w in enumerate(self.words)}
      self.itos_ = {idx: w for w, idx in self.stoi_.items()}

    def itos(self, idx):
        """Возвращает токен по индексу"""
        return self.itos_.get(idx, '<UNK>')

    def stoi(self, s):
        """Возвращает индекс токена"""
        return self.stoi_.get(s, 1) # self.stoi_['<UNK>'])

In [ ]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]

In [ ]:
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt_tab')

tokenizer = RegexpTokenizer(r"\w+")

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
corpus_tokens = [
    tokenizer.tokenize(doc.lower())
    for doc in corpus
]
corpus_tokens

[['маленький', 'котенок', 'игриво', 'прыгает', 'за', 'шариком'],
 ['пушистый', 'котик', 'мурлыкает', 'лежа', 'на', 'солнышке'],
 ['котенок', 'любопытно', 'нюхает', 'цветы', 'в', 'саду'],
 ['котик', 'ловко', 'лазает', 'по', 'дереву', 'исследуя', 'окружающий', 'мир'],
 ['спящий', 'котик', 'мило', 'моргает', 'своими', 'яркими', 'глазками']]

In [ ]:
vocab = Vocab(corpus_tokens)

encoded_corpus = [[vocab.stoi(t) for t in doc] for doc in corpus_tokens]
encoded_corpus

[[6, 24, 5, 16, 18, 31],
 [1, 4, 12, 30, 29, 8],
 [24, 25, 20, 33, 0, 2],
 [4, 15, 21, 7, 13, 22, 32, 17],
 [10, 4, 28, 26, 14, 27, 19]]

In [ ]:
vocab.stoi('мурлыкает')

12

In [ ]:
vocab.stoi('москва')

1

In [ ]:
vocab.itos(7)

'по'

In [ ]:
vocab.itos(100)

'<UNK>'

### 2.

<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте собственную реализацию `Vocab`. Набор индексов токенов возвращайте в виде тензора. Предусмотрите возможность передавать в конструктор класса преобразования.

Выведите на экран результат выполнения `dataset[0]`.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd

pd.read_csv('news.csv')

,text,label
0,лукашенко пригрозил литовским танкам белорусск...,0
1,российские компании оказались в опасности из з...,1
2,лукашенко объявил об отмене выборов из за напа...,0
3,«роснефть» стала лидером по объему биржевых пр...,1
4,псковской области дадут десятки миллионов рубл...,1
...,...,...
1096,борис джонсон пообещал избирателям вернуть ирл...,0
1097,михаил горбачев заявил что не подавал в отстав...,0
1098,в мгимо появится факультет международных отнош...,0
1099,в россии введут штраф за отрицание агрессии нато,0


In [ ]:
pd.read_csv('news.csv').iloc[0]['text']

'лукашенко пригрозил литовским танкам белорусской картошкой '

In [ ]:
class NewsDataset:
  def __init__(self, f, vocab=None, tokenizer=None, transform = None):
    self.data = pd.read_csv(f)
    self.tokenizer = tokenizer or RegexpTokenizer(r'\w+')
    self.vocab = vocab or self.create_vocab(self.data['text']) #vocab if vocab is not None else
    self.transform = transform

  def create_vocab(self, texts):
    corpus_tokens = [self.tokenizer.tokenize(t.lower()) for t in texts]
    return Vocab(corpus_tokens)

  def __getitem__(self, idx):
    text = self.data.iloc[idx]['text']
    label = self.data.iloc[idx]['label']

    tokens = self.tokenizer.tokenize(text.lower())
    idxs = [self.vocab.stoi(t) for t in tokens]

    if self.transform:
      idxs = self.transform(idxs)

    return th.tensor(idxs), label


dataset = NewsDataset('news.csv')
tokens, label = dataset[0]
print(tokens)
print('label:', label)

tensor([ 977, 5191, 2126, 1997, 3835, 2124])
label: 0


### 3.

<p class="task" id="3"></p>

3\. Реализуйте преобразование `Truncate`, которое обрезает каждый текст в батче до `n` токенов. Создайте объект `NewsDataset` с обрезкой предложений до 5 токенов (указав данное преобразование при создании объекта). Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

In [ ]:
class Truncate:
    def __init__(self, n):
      self.n = n

    def __call__(self, token):
      return token[:self.n]

In [ ]:
class NewsDataset:
  def __init__(self, f, vocab=None, tokenizer=None, transform = None):
    self.data = pd.read_csv(f)
    self.tokenizer = tokenizer or RegexpTokenizer(r'\w+')
    self.vocab = vocab or self.create_vocab(self.data['text']) #vocab if vocab is not None else
    self.transform = transform

  def create_vocab(self, texts):
    corpus_tokens = [self.tokenizer.tokenize(t.lower()) for t in texts]
    return Vocab(corpus_tokens)

  def __getitem__(self, idx):
    text = self.data.iloc[idx]['text']
    label = self.data.iloc[idx]['label']

    tokens = self.tokenizer.tokenize(text.lower())
    idxs = [self.vocab.stoi(t) for t in tokens]

    if self.transform:
      idxs = self.transform(idxs)

    return th.tensor(idxs), label

  def __len__(self):  ## добавила
    return len(self.data)

trunc = Truncate(5)
dataset = NewsDataset('news.csv', transform = trunc)
tokens, label = dataset[0]
print(tokens)
print('label:', label)

tensor([ 977, 5191, 2126, 1997, 3835])
label: 0


In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for batch in dataloader:
  tokens, labels = batch
  print(tokens, tokens.shape)
  print(labels, labels.shape)
  break

tensor([[5397, 5103, 2286, 3864,  524],
        [2596, 1421, 3718,  385, 4465],
        [3258,  730, 3982, 1489, 5366],
        [4195, 3758, 3718, 4948,  418],
        [1564, 4901, 1310, 1114, 3718],
        [4400, 5782, 3688, 4082, 1445],
        [2808,  833, 2638,  463, 1836],
        [2434,  290,  478, 5160,  241],
        [1445,  103, 4454, 1065, 3780],
        [3716, 5211,  366, 3587, 3982],
        [4951, 3718, 1026, 2733, 5040],
        [3183, 3658, 3469, 2095,  332],
        [1445, 4973, 1324, 3643,  694],
        [1051, 2348, 2036, 4607, 1859],
        [4400, 2808, 1445, 3576, 3664],
        [4132, 3018,  562, 1445, 3743]]) torch.Size([16, 5])
tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]) torch.Size([16])


### 4.

<p class="task" id="4"></p>

4\. Реализуйте преобразование `Pad`, которое расширяет каждый текст в батче до `n` токенов значением `pad_idx`. Создайте объект `NewsDataset` с расширением предложений до 30 токенов (указав данное преобразование при создании объекта). Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

In [ ]:
class Pad:
    def __init__(self, n, pad_idx):
      self.n = n
      self.pad_idx = pad_idx

    def __call__(self, token):
      return token + [self.pad_idx]*(self.n-len(token))

In [ ]:
dataset = NewsDataset('news.csv')
pad_idx = dataset.vocab.stoi('<PAD>')
pad_idx

4092

In [ ]:
pad = Pad(30, pad_idx)
dataset = NewsDataset('news.csv', transform = pad)
tokens, label = dataset[0]

print(tokens)
print('label:', label)

tensor([ 977, 5191, 2126, 1997, 3835, 2124, 4092, 4092, 4092, 4092, 4092, 4092,
        4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
        4092, 4092, 4092, 4092, 4092, 4092])
label: 0


In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for batch in dataloader:
  texts, labels = batch
  print(texts, texts.shape)
  print(labels, labels.shape)
  break

tensor([[4701, 2537,  510, 5356, 1445, 4989, 3337, 4692, 2733, 3748, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092],
        [1445, 1577, 3631,  292, 4050, 4060, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092],
        [1445,  312, 4758, 5266, 5453,  917, 1510, 2733, 2540, 5958, 2105, 5707,
         1071, 1449,  583, 1510, 1652,  168, 3101, 4216, 3386, 4699, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092],
        [2284, 2203, 1908,  201, 3256, 4719, 3117, 1445, 2403, 2807, 3341, 4775,
         2469, 4394, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092],
        [2833, 1537, 2648, 5721, 3273, 1859,  508, 3092, 3570, 4804, 3943, 4092,
         4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
      

### 5.

<p class="task" id="5"></p>

5\. Создайте объект `tokenizers.Tokenizer` на основе данных из файла `news.csv`. Для выделения токенов из текста используйте разбиение по пробелам (pre-tokenizer `Whitespace` + model `WordLevel` ). На этапе нормализации приводите текст к нижнему регистру и убирайте знаки препинания. Включите опцию паддинга. Первые 4 индекса зарезервируйте под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.

Опишите класс `NewsDatasetHfTokenizer`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте обученный токенизатор. Набор индексов токенов возвращайте в виде тензора. Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordLevelTrainer

In [ ]:
data = pd.read_csv("news.csv")

In [ ]:
corpus = data.text.to_list()
corpus[:5]

['лукашенко пригрозил литовским танкам белорусской картошкой ',
 'российские компании оказались в опасности из за глобального потепления',
 'лукашенко объявил об отмене выборов из за нападения иностранных диверсантов ',
 '«роснефть» стала лидером по объему биржевых продаж нефтепродуктов',
 'псковской области дадут десятки миллионов рублей на ремонт дорог']

In [ ]:
model = WordLevel()  # разбиваем на слова
tokenizer = Tokenizer(model)
tokenizer.normalizer = Lowercase() # к нижнему регистру
tokenizer.pre_tokenizer = Whitespace()  # токен-ция по пробелам

trainer = WordLevelTrainer(special_tokens=["<PAD>", "<UNK>", "<SOS>", "<EOS>"])
tokenizer.train_from_iterator(corpus, trainer=trainer)  # обучаем ток-р на корпусе т

In [ ]:
pad_id = tokenizer.token_to_id("<PAD>")
pad_id

0

In [ ]:
tokenizer.token_to_id("<UNK>")

1

In [ ]:
max_len = max(len(t) for t in corpus)
max_len

165

In [ ]:
tokenizer.enable_padding(pad_id = pad_id, pad_token="<PAD>", length=max_len) # включаем паддинг


Опишите класс `NewsDatasetHfTokenizer`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте обученный токенизатор. Набор индексов токенов возвращайте в виде тензора. Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.


In [ ]:
class NewsDatasetHfTokenizer:
    def __init__(self, f, tokenizer, transform=None):
        self.data = pd.read_csv(f)
        self.tokenizer = tokenizer
        self.transform = transform

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        label = self.data.iloc[idx]["label"]

        tokens = self.tokenizer.encode(text)  # токенизируем (ч/з .tokens)
        idxs = tokens.ids

        if self.transform:
            idxs = self.transform(idxs)

        return th.tensor(idxs), label

    def __len__(self):
        return len(self.data)

In [ ]:
def collate_fn(batch):        # вручную паддим
  text, labels = zip(*batch)  # распакоука на
  max_len = max(len(t) for t in texts)
  padded_texts = [th.cat([t, th.full((max_len - len(text),), pad_id)]) for t in texts]
  # обзор чеек : full - создаем тенз нужной лен и заполняем pad_id
  # cat - конкатенируем, в итоге получаем западденные тексты :)

  return th.stack(padded_texts), th.tensor(labels)
  # ну стакаем типо до 1 тенза

In [ ]:
dataset = NewsDatasetHfTokenizer("news.csv", tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

for batch in dataloader:
  texts, labels = batch
  print(texts, texts.shape)
  print(labels, labels.shape)
  break

tensor([[4701, 2537,  510, 5356, 1445, 4989, 3337, 4692, 2733, 3748, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [1445, 1577, 3631,  292, 4050, 4060, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [1445,  312, 4758, 5266, 5453,  917, 1510, 2733, 2540, 5958, 2105, 5707,
         1071, 1449,  583, 1510, 1652,  168, 3101, 4216, 3386, 4699, 4092, 4092,
         4092, 4092, 4092, 4092, 4092, 4092,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [2284, 2203, 1908,  201, 3256, 4719, 3117, 1445, 2403, 2807, 3341, 4775,
         2469, 4

In [ ]:
# если с enable_padding (без ручного падд),
# то размер батча будет максимальным по длине и не будет учитывать макс лен у 1 батча (ну многа памяти тип, не оч эффективно):

In [ ]:
dataset = NewsDatasetHfTokenizer("news.csv", tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for batch in dataloader:
  texts, labels = batch
  print(texts, texts.shape)
  print(labels, labels.shape)
  break

tensor([[  47, 4961,  179,  ...,    0,    0,    0],
        [4156,   97, 4132,  ...,    0,    0,    0],
        [1953, 4225,  953,  ...,    0,    0,    0],
        ...,
        [ 100,   64,   11,  ...,    0,    0,    0],
        [   7, 3436, 1202,  ...,    0,    0,    0],
        [ 331,   68, 1384,  ...,    0,    0,    0]]) torch.Size([16, 165])
tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]) torch.Size([16])
